In [1]:
!pip install -Uq tensorflow_hub  pydub

In [2]:
!pip install -Uq tensorflow-io

In [3]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the YAMNET model
model = hub.load('https://tfhub.dev/google/yamnet/1')

# Load an audio file
audio, sample_rate = tf.audio.decode_wav(tf.io.read_file('audio/h1.wav'))
audio = tf.squeeze(audio, axis=-1)

# Generate embeddings
scores, embeddings, log_mel_spectrogram = model(audio)

print(f"Audio embedding shape: {embeddings.shape}")

2024-10-09 13:41:25.495832: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-10-09 13:41:25.495854: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-10-09 13:41:25.495860: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-10-09 13:41:25.495877: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-09 13:41:25.495884: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-10-09 13:41:26.761307: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Audio embedding shape: (34, 1024)


In [4]:
# Load an audio file
audio, sample_rate = tf.audio.decode_wav(tf.io.read_file('audio/a2.wav'))
audio = tf.squeeze(audio, axis=-1)

scores, embeddings, log_mel_spectrogram = model(audio)

print(f"Audio embedding shape: {embeddings.shape}")#27,18

Audio embedding shape: (41, 1024)


In [5]:
log_mel_spectrogram

<tf.Tensor: shape=(2016, 64), dtype=float32, numpy=
array([[-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       ...,
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554],
       [-6.9077554, -6.9077554, -6.9077554, ..., -6.9077554, -6.9077554,
        -6.9077554]], dtype=float32)>

#  we found shape dimension error
* you can solve with slicing

In [17]:
import os
import numpy as np

voices = []
labels = []
for i in os.listdir("./audio/"):
    if '.wav' in i:
        name = i.split(".")[0]


        audio, sample_rate = tf.audio.decode_wav(tf.io.read_file(f'./audio/{i}'))
        audio = tf.squeeze(audio, axis=-1)

        scores, embeddings, log_mel_spectrogram = model(audio)

        voices.append(np.array(embeddings[:5,:]).ravel())
        labels.append(name)

        print(f"Audio embedding shape: {embeddings.shape} new shape {embeddings[:5,:].shape} type {type(np.array(embeddings[:18,:]))}")#27,18
        print(i)

Audio embedding shape: (41, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
a2.wav
Audio embedding shape: (29, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
a1.wav
Audio embedding shape: (34, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
h1.wav
Audio embedding shape: (18, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
q2.wav
Audio embedding shape: (42, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
h2.wav
Audio embedding shape: (27, 1024) new shape (5, 1024) type <class 'numpy.ndarray'>
q1.wav


In [18]:
voices

[array([1.6946361 , 0.36075723, 0.6930015 , ..., 0.9783664 , 0.12747997,
        0.        ], dtype=float32),
 array([0.        , 0.25089428, 0.09453958, ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([2.2192965 , 0.14239739, 0.6774485 , ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([2.103551  , 0.28917295, 0.66041994, ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([1.8627615 , 0.10154425, 0.7127788 , ..., 0.2883861 , 1.3826851 ,
        2.7965832 ], dtype=float32),
 array([2.324272  , 0.17074186, 0.21542566, ..., 0.0181713 , 1.7490327 ,
        1.825291  ], dtype=float32)]

In [27]:
#Imports a PyMilvus package:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

#Connect to the Milvus
connections.connect("default", host="localhost", port="19530")
# Define the collection name
collection_name = "audio"

# Delete old collection if it exists
if utility.has_collection(collection_name):
    Collection(collection_name).drop()
    
    
#Creates a collection:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="words", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="person_name", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=5120)
]
schema = CollectionSchema(fields, "Simple Demo for audio similar search")
audio = Collection("audio", schema)


In [28]:
# Builds indexes on the entities:

index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

audio.create_index("embeddings", index)

Status(code=0, message=)

In [29]:
labels

['a2', 'a1', 'h1', 'q2', 'h2', 'q1']

In [30]:
voices[0].shape

(5120,)

In [31]:
#Insert data in collection
data = [
    [1,2,3,4,5,6],  # field pk
    labels,  # field words
    ["Auranzaib","Auranzaib","Hasnant","Qasim","Hasnant","Qasim"],
    voices,  # field embeddings
]

In [32]:
audio.insert(data)
audio.flush()
audio.load()

In [34]:
search_params = {"metric_type": "L2"}

In [35]:
results = audio.search(
	data=[voices[0]],
	anns_field="embeddings",
	param=search_params,
	limit=4,
	expr=None,
	# set the names of the fields you want to retrieve from the search result.
	output_fields=['words','person_name'],
	consistency_level="Strong"
)

In [37]:
for i in range(0,len(results[0])):
    name = results[0][i].entity.get('words')
    pname = results[0][i].entity.get('person_name')
    print(pname)
#     try:
#         display(Image.open('./images/'+name+'.jpg'))
#     except:
#         display(Image.open('./images/'+name+'.png'))

Auranzaib
Auranzaib
Qasim
Hasnant


In [38]:
labels

['a2', 'a1', 'h1', 'q2', 'h2', 'q1']